In [92]:
import torch

def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index],
                      db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

def query(db, threshold=5):
    return (db.sum() > threshold).float()

def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
            
    return sensitivity

In [93]:
db, pdbs = create_db_and_parallels(100)
db

tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 1], dtype=torch.uint8)

In [94]:
true_result = torch.mean(db.float())
true_result

tensor(0.4600)

In [95]:
def query(db, noise=0.5):
    public_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) < noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augumented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
    sk_result = torch.mean(augumented_database.float())

    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)
    
    return private_result, public_result

In [96]:
result = torch.rand(5)
result

tensor([0.4489, 0.4957, 0.8672, 0.0152, 0.5104])

In [97]:
result < 1

tensor([1, 1, 1, 1, 1], dtype=torch.uint8)

In [98]:
db, pdbs = create_db_and_parallels(100)
private_result, public_result = query(db, noise=0.2)
print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.5250)
Without noise:tensor(0.4800)


In [99]:
db, pdbs = create_db_and_parallels(100)
private_result, public_result = query(db, noise=0.4)
print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.4000)
Without noise:tensor(0.4900)


In [100]:
db, pdbs = create_db_and_parallels(100)
private_result, public_result = query(db, noise=0.6)
print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.5750)
Without noise:tensor(0.5000)


In [101]:
db, pdbs = create_db_and_parallels(100)
private_result, public_result = query(db, noise=0.8)
print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.6000)
Without noise:tensor(0.5100)


In [102]:
db, pdbs = create_db_and_parallels(10000)
private_result, public_result = query(db, noise=0.8)
print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.4625)
Without noise:tensor(0.4915)
